In [2]:
import pandas as pd
import pyperclip
import pyautogui
import keyboard
from time import sleep
from pywinauto import Application
from pywinauto.mouse import click
from pywinauto.keyboard import send_keys

# --- Configuración ---
file_path = r"C:\Users\Administrator\DMV Automation - 10 Jun\Archivos Consolidados\testUser - Consolidado.xlsx"
sheet_name = "Partitions"
EDITOR_COORD = (1581, 1387)
TEXT_RESULT_COORD = (1581, 1387)

# --- Leer hoja Partitions ---
df = pd.read_excel(file_path, sheet_name=sheet_name)

# --- Agregar lógica DAX para "Source Type" ---
def calcular_source_type(query: str) -> str:
    if not isinstance(query, str):
        return "Other"
    clean = query.replace('\n', '').strip()
    if any(clean.startswith(p) for p in [
        "CALCULATETABLE", "SELECTCOLUMNS", "VAR", "UNION",
        "DISTINCT", "SUMMARIZE", "FILTER", "CALENDAR"
    ]):
        return "DAX"
    if "AnalysisServices.Database" in clean:
        return "Analysis Services"
    if "ActiveDirectory" in clean:
        return "Active Directory"
    if "Excel.Workbook" in clean:
        return "Excel Workbook"
    if "Csv.Document" in clean:
        return "CSV Document"
    if "mySQL.Database" in clean:
        return "MySQL Database"
    if "Oracle.Database" in clean:
        return "Oracle Database"
    if "OData.Feed" in clean:
        return "OData Feed"
    if "Sharepoint" in clean:
        return "SharePoint"
    if "Table.FromRows" in clean:
        return "Manual Input"
    return "Other"

# --- Calcular si no existe ---
if "Source Type" not in df.columns:
    df["Source Type"] = df["QueryDefinition"].apply(calcular_source_type)

# --- Inicializar columnas ---
if "QueryFormattedShort" not in df.columns:
    df["QueryFormattedShort"] = ""
if "QueryFormattedLong" not in df.columns:
    df["QueryFormattedLong"] = ""

# --- Filtrar solo DAX ---
dax_df = df[df["Source Type"] == "DAX"].copy()

# --- Conectar a DAX Studio ---
app = Application(backend="uia").connect(title_re=".*DAX Studio.*")
win = app.window(title_re=".*DAX Studio.*")
win.set_focus()
sleep(0.2)

# --- Iterar solo sobre casos DAX ---
for i in dax_df.index:
    if keyboard.is_pressed('p'):
        print("\n⛔️ Abortado con tecla 'P'")
        break

    query = df.at[i, "QueryDefinition"]
    if not isinstance(query, str) or query.strip() == "":
        continue

    print(f"[{i+1}] Formateando...")

    # Copiar, pegar
    pyperclip.copy(query)
    click(coords=EDITOR_COORD)
    send_keys("^a{BACKSPACE}")
    sleep(0.05)
    send_keys("^v")
    sleep(0.05)

    # --- Short (Ctrl + F6) ---
    pyautogui.hotkey('ctrl', 'f6')
    sleep(0.4)
    click(coords=TEXT_RESULT_COORD)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.hotkey('ctrl', 'c')
    sleep(0.05)
    df.at[i, "QueryFormattedShort"] = pyperclip.paste()

    # --- Long (F6) ---
    pyautogui.press('f6')
    sleep(0.4)
    click(coords=TEXT_RESULT_COORD)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.hotkey('ctrl', 'c')
    sleep(0.05)
    df.at[i, "QueryFormattedLong"] = pyperclip.paste()

    print(f"[{i+1}] ✅ OK")

# --- Guardar en la misma hoja del archivo ---
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ Hoja 'Partitions' actualizada en: {file_path}")


[4] Formateando...
[4] ✅ OK
[12] Formateando...
[12] ✅ OK
[21] Formateando...
[21] ✅ OK
[26] Formateando...
[26] ✅ OK
[54] Formateando...
[54] ✅ OK
[62] Formateando...
[62] ✅ OK
[71] Formateando...
[71] ✅ OK
[75] Formateando...
[75] ✅ OK
[83] Formateando...
[83] ✅ OK
[92] Formateando...
[92] ✅ OK

✅ Hoja 'Partitions' actualizada en: C:\Users\Administrator\DMV Automation - 10 Jun\Archivos Consolidados\testUser - Consolidado.xlsx
